In [ ]:
import numpy as np
from keras import callbacks
from keras.callbacks import History
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import utils
import string
from typing import List, Tuple
import re

In [ ]:
import pandas as pd
from pandas import DataFrame


def load_lyrics(csv_path: str) -> DataFrame:
    df = pd.read_csv(csv_path, sep='\n', header=None)
    res = df.iloc[:, 0].str.rstrip(r'&, ').str.extract(r'([^,]+),([^,]+),(.+)')
    res.columns = ['artist', 'title', 'lyrics']
    return res

def load_songs(path) -> List[str]:
    df = load_lyrics(path)
    songs = []
    for i, song in enumerate(list(df['lyrics'])):
        song += " EOF"
        modified_song = re.sub(r"\([^()]*\)", "", song)
        # modified_song = re.sub(r"\[[^()]*\]", "", modified_song)
        # for ch in ["[", "]", "chorus"]:
        #     modified_song = modified_song.replace(ch, "")
        modified_song = modified_song.replace("chorus", "").replace("&", "silencio").lower()
        # modified_song = modified_song.replace("&", "silencio").lower()
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        modified_song = regex.sub('', modified_song)
        df.loc[i, 'lyrics'] = modified_song
        songs.append(modified_song)
    return songs, df

In [ ]:
def create_x_y_train(songs, tokenize, total_words) -> Tuple[np.ndarray, np.ndarray]:
    input_sequences = []
    midi_meta = []
    for index, row in songs.iterrows():
      line = row['lyrics']
    # for line in songs:
      token_list = tokenize.texts_to_sequences([line])[0]
      midi_metadata = row.drop(['artist', 'title', 'lyrics', 'filename']).values
      for i in range(1, len(token_list)):
          n_gram_sequence = token_list[:i + 1]
          input_sequences.append(n_gram_sequence)
          midi_meta.append(midi_metadata)
    print(input_sequences[:10])
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='post'))
    x_train = input_sequences[:, :-1]
    x_train_midi = np.array(midi_meta) #midi_meta[:, :-1]
    y_train = input_sequences[:, -1]
    y_train = utils.to_categorical(y_train, num_classes=total_words)
    return x_train, x_train_midi, y_train, max_sequence_len

In [ ]:
class LyricsGenerator(object):

    def __init__(self, embedding_dim: int, vocab_size: int, input_size: int, midi_features_size: int, embedding_matrix: np.ndarray):
        input_lyrics = Input(shape=(input_size,))
        embedding_layer = Embedding(
            vocab_size,
            embedding_dim,
            input_length=input_size,
            # input_shape=(vocab_size, ),
            weights=[embedding_matrix],
            # embeddings_initializer=initializers.Constant(embedding_matrix),
            trainable=False,
        )(input_lyrics)
        lstm = LSTM(units=embedding_dim)(embedding_layer)
        lyrics_model = Dropout(0.2)(lstm)

        # self.lyrics = Sequential()
        # self.lyrics.add(embedding_layer)
        # self.lyrics.add(LSTM(units=embedding_dim))
        # self.lyrics.add(Dropout(0.2))

        # self.midi = Sequential()
        # self.midi.add(Dense(50, input_shape=(midi_features_size,), activation='relu'))
        input_midi =  Input(shape=(midi_features_size,))
        midi_model = Dense(50, input_shape=(midi_features_size,), activation='relu')(input_midi)

        # self.model = Sequential()
        final_model_1 = concatenate([lyrics_model, midi_model])
        final_model = Dense(units=vocab_size, activation='softmax')(final_model_1)
        # self.model.add(Dense(units=vocab_size, activation='softmax'))
        self.model = Model(inputs=[input_lyrics, input_midi], outputs=final_model)
        self.model.compile(optimizer='adam', loss='categorical_crossentropy')
        # self.model.build(input_shape)
        self.model.summary()

    def fit(self, x, y, hyper_parameters):
        callback = callbacks.EarlyStopping(monitor='loss', patience=3)
        callback2 = callbacks.LearningRateScheduler(self._lr_scheduler)
        # callback3 = callbacks.ModelCheckpoint('lyrics_model.h5', save_best_only=True, monitor='val_loss', mode='min')
        # callback = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=1)
        # history = self.model.fit(x, y, epochs=100, verbose=1)
        history = self.model.fit(x, y, batch_size=hyper_parameters['batch_size'], epochs=hyper_parameters['epochs'],
                                 callbacks=[callback, callback2],
                                 verbose=1, validation_split=hyper_parameters['validation_split'],
                                 validation_data=None)
        self.model.save('lyrics_model.h5')
        
        return history

    def _lr_scheduler(self, epoch, lr):
        return 0.99 * lr

    # to get a picture of loss progress.
    def plot_metric(self, history: History, metric: str = 'loss') -> None:
        import matplotlib.pyplot as plt
        train_metrics = history.history[metric]
        val_metrics = history.history['val_'+metric]
        epochs = range(1, len(train_metrics) + 1)
        plt.plot(epochs, train_metrics)
        plt.plot(epochs, val_metrics)
        plt.title('Training and validation '+ metric)
        plt.xlabel("Epochs")
        plt.ylabel(metric)
        plt.legend(["train_"+metric, 'val_'+metric])
        plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
songs, df_songs = load_songs('/content/drive/MyDrive/DL/Assignment_3/Lyrics/lyrics_train_set.csv')

In [ ]:
songs[0]

'goodbye norma jean silencio though i never knew you at all silencio you had the grace to hold yourself silencio while those around you crawled silencio they crawled out of the woodwork silencio and they whispered into your brain silencio they set you on the treadmill silencio and they made you change your name silencio and it seems to me you lived your life silencio like a candle in the wind silencio never knowing who to cling to silencio when the rain set in silencio and i would liked to have known you silencio but i was just a kid silencio your candle burned out long before silencio your legend ever did silencio loneliness was tough silencio the toughest role you ever played silencio hollywood created a superstar silencio and pain was the price you paid silencio even when you died silencio oh the press still hounded you silencio all the papers had to say silencio was that marilyn was found in the nude silencio and it seems to me you lived your life silencio like a candle in the wind

In [ ]:
# embedding_matrixs =  np.load("/content/drive/MyDrive/DL/Assignment_3/Lyrics/embedding_matrix.npy")

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DL/Assignment_3/midi_files.zip',"r") as zip_ref:
  zip_ref.extractall("temp_dir")

In [ ]:
!pip install MIDIFile

In [ ]:
from os import listdir
from MIDI import MIDIFile


files_in_dir = [f for f in listdir("temp_dir/midi_files") if re.match('.*\.mid', f)]
df_midi = pd.DataFrame(columns=['filename', 'numerator', 'denominator', 'clocksPerTick', 'demisemiquaverPer24Clocks', 'key', 'mode', 'tempo'])
def parse(file):
    c=MIDIFile(file)
    c.parse()
    numerator = denominator = clocksPerTick = demisemiquaverPer24Clocks = -1
    key = mode = 'unknown'
    tempo = -1
    for idx, track in enumerate(c):
        track.parse()
        data = str(track)
        if 'Time Signature -> ' in data and numerator == -1:
          t_s = data.split('Time Signature -> ')[1].split('\n',1)[0].split(" ")
          numerator = t_s[0].split("numerator=")[1]
          denominator = t_s[1].split("denominator=")[1]
          clocksPerTick = t_s[2].split("clocksPerTick=")[1]
          demisemiquaverPer24Clocks = t_s[3].split("demisemiquaverPer24Clocks=")[1]

        if 'Key Signature -> ' in data and key == 'unknown':
          k_s = data.split('Key Signature -> ')[1].split('\n',1)[0].split(" ")
          key = k_s[0].split("key=")[1]
          mode = k_s[1].split("mode=")[1]

        if 'Set Tempo -> tempo=' in data and tempo == -1:
          tempo = data.split('Set Tempo -> tempo=')[1].split('\n',1)[0]
    
    return numerator,denominator,clocksPerTick,demisemiquaverPer24Clocks,key,mode,tempo #str(track)
for i, f in enumerate(files_in_dir):

    try:
      numerator,denominator,clocksPerTick,demisemiquaverPer24Clocks,key,mode,tempo = parse("temp_dir/midi_files/" + f)
    except UnicodeDecodeError as e:
      continue
    
    print(f'n {numerator} d {denominator} cpt {clocksPerTick} dp24c {demisemiquaverPer24Clocks}')
    print(f'key {key} mode {mode} tempo {tempo}')
    df_tmp = pd.DataFrame([[f.lower(),numerator,denominator,clocksPerTick,demisemiquaverPer24Clocks,key,mode,tempo ]], columns=['filename', 'numerator', 'denominator', 'clocksPerTick', 'demisemiquaverPer24Clocks', 'key', 'mode', 'tempo'])
    df_midi= pd.concat([df_midi, df_tmp])


n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 175.60968
n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 145.45464
n 4 d 4 cpt 24 dp24c 8
key F# mode major tempo 156.52176


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/MIDI/chunks/track.py", line 37, in parse
    event = MIDIEvent(time,self.buffer)
  File "/usr/local/lib/python3.7/dist-packages/MIDI/Events/midi.py", line 43, in __init__
    length = len(self.message) if self.message else 0
TypeError: 'method' object cannot be interpreted as an integer


n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 130.9092
n -1 d -1 cpt -1 dp24c -1
key unknown mode unknown tempo -1
n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 138.46152
n -1 d -1 cpt -1 dp24c -1
key C mode major tempo 192.0
n 4 d 4 cpt 24 dp24c 8
key unknown mode unknown tempo 180.0
n -1 d -1 cpt -1 dp24c -1
key unknown mode unknown tempo 112.5
n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 144.0


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/MIDI/chunks/track.py", line 30, in parse
    eventType=self.buffer[0]
IndexError: index out of range
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/MIDI/chunks/track.py", line 37, in parse
    event = MIDIEvent(time,self.buffer)
  File "/usr/local/lib/python3.7/dist-packages/MIDI/Events/midi.py", line 43, in __init__
    length = len(self.message) if self.message else 0
TypeError: 'method' object cannot be interpreted as an integer


n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 90.0
n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 141.1764
n -1 d -1 cpt -1 dp24c -1
key unknown mode unknown tempo 142.57416
n 4 d 4 cpt 24 dp24c 8
key G mode major tempo 144.0
n -1 d -1 cpt -1 dp24c -1
key unknown mode unknown tempo -1


...

n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 82.28568
n -1 d -1 cpt -1 dp24c -1
key A mode major tempo 96.0


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/MIDI/chunks/track.py", line 30, in parse
    eventType=self.buffer[0]
IndexError: index out of range


n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 114.2856
n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 293.8776
n -1 d -1 cpt -1 dp24c -1
key unknown mode unknown tempo 240.0


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/MIDI/chunks/track.py", line 30, in parse
    eventType=self.buffer[0]
IndexError: index out of range
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/MIDI/chunks/track.py", line 30, in parse
    eventType=self.buffer[0]
IndexError: index out of range


n -1 d -1 cpt -1 dp24c -1
key C mode major tempo 72.0
n 4 d 4 cpt 24 dp24c 8
key C mode major tempo 151.57896
n -1 d -1 cpt -1 dp24c -1
key C mode major tempo 130.90896
n -1 d -1 cpt -1 dp24c -1
key unknown mode unknown tempo -1
n -1 d -1 cpt -1 dp24c -1
key C mode major tempo 90.0


In [ ]:
df_midi

,filename,numerator,denominator,clocksPerTick,demisemiquaverPer24Clocks,key,mode,tempo
0,boyzone_-_i_love_the_way_you_love_me.mid,4,4,24,8,C,major,175.60968
0,the_eagles_-_the_long_run.mid,4,4,24,8,C,major,145.45464
0,bee_gees_-_nights_on_broadway.mid,4,4,24,8,F#,major,156.52176
0,the_eagles_-_tequila_sunrise.mid,4,4,24,8,C,major,130.9092
0,christmas_carols_-_rockin_around_the_christmas...,-1,-1,-1,-1,unknown,unknown,-1
...,...,...,...,...,...,...,...,...
0,culture_club_-_karma_chameleon.mid,-1,-1,-1,-1,C,major,72.0
0,cypress_hill_-_rock_superstar.mid,4,4,24,8,C,major,151.57896
0,genesis_-_afterglow.mid,-1,-1,-1,-1,C,major,130.90896
0,2_unlimited_-_get_ready_for_this.mid,-1,-1,-1,-1,unknown,unknown,-1


In [ ]:
df_songs['filename'] = df_songs['artist'].str.replace(' ', "_") + "_-_" + df_songs['title'].str.replace(' ', "_") + ".mid"
df_merged = pd.merge(df_songs, df_midi, on="filename")
#df_merged.to_csv('/content/drive/MyDrive/DL/Assignment_3/songs.csv' ,index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

scalers = []
columns = ['numerator', 'denominator','clocksPerTick', 'demisemiquaverPer24Clocks', 'tempo']
for col in columns:
  scaler = MinMaxScaler()
  scaled_features = scaler.fit_transform(df_merged[col].values.reshape(-1, 1))
  tmp_df = pd.DataFrame(scaled_features, index=df_merged.index, columns=[col])
  df_merged[col] = tmp_df[col]
  scalers.append(scaler)


onehot_encoders = []
columns = ['key', 'mode']
for col in columns:
  onehot_encoder = OneHotEncoder(sparse=False)
  scaled_features = onehot_encoder.fit_transform(df_merged[col].values.reshape(-1, 1))
  tmp_df = pd.DataFrame(scaled_features, index=df_merged.index)
  tmp_df.columns = onehot_encoder.get_feature_names([col])
  df_merged.drop([col] ,axis=1, inplace=True)
  df_merged = pd.concat([df_merged, tmp_df ], axis=1)
  onehot_encoders.append(onehot_encoder)
  

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/DL/Assignment_3/Lyrics/lyrics_train_set.csv', sep='\n', header=None)
# res = df.iloc[:, 0].str.rstrip(r'&, ').str.extract(r'([^,]+),([^,]+),(.+)')
# print(len(res))
# res.columns = ['artist', 'title', 'lyrics']
# print(len(res['lyrics']))

# songs = []
# for song in list(res['lyrics']):
#     song += " EOF"
#     modified_song = re.sub(r"\([^()]*\)", "", song)
#     modified_song = modified_song.replace("chorus", "").replace("&", "silencio").lower()
#     regex = re.compile('[%s]' % re.escape(string.punctuation))
#     modified_song = regex.sub('', modified_song)
#     songs.append(modified_song)

# len(songs)

In [ ]:
# import tensorflow as tf
# import random
# tf.random.set_seed(1)
# random.seed(2)
# np.random.seed(3)

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DL/Assignment_3/glove.6B.zip',"r") as zip_ref:
  zip_ref.extractall("glove")

In [ ]:
# Data preprocessing
tokenize = Tokenizer()
tokenize.fit_on_texts(df_merged['lyrics'])
total_words = len(tokenize.word_index) + 1

embedding_dim = 300
x_train, x_train_midi, y_train, max_sequence_len = create_x_y_train(df_merged, tokenize, total_words)

[[220, 2434], [220, 2434, 2435], [220, 2434, 2435, 1], [220, 2434, 2435, 1, 320], [220, 2434, 2435, 1, 320, 4], [220, 2434, 2435, 1, 320, 4, 47], [220, 2434, 2435, 1, 320, 4, 47, 224], [220, 2434, 2435, 1, 320, 4, 47, 224, 2], [220, 2434, 2435, 1, 320, 4, 47, 224, 2, 68], [220, 2434, 2435, 1, 320, 4, 47, 224, 2, 68, 19]]


In [ ]:
import pickle
f = open("tokenizer_pickle", "wb")
pickle.dump(tokenize, f)
f.close()

In [ ]:
#taken from https://gist.github.com/shubham0204/531f75c37c49362c9b61a11865496177
import numpy as np
import pickle
import os

# You need to download the GloVe embeddings ( in txt files ) from here -> http://nlp.stanford.edu/data/glove.6B.zip

# The vocabulary size. It is the value of `len( tokenizer.word_index )+1`
vocab_size = total_words #int(input('Enter Vocabulary Size : '))

# The file to the GloVe file. For instance, "glove.6B/glove.6B.50d" where "50" represents the output dimension of the embedding.
glove_path = "/content/glove/glove.6B.300d.txt" #input('Enter path to GloVe text file : ')

# The filepath of the tf.keras.preprocessing.text.Tokenizer object in pickled form. 
# This object was earlier used to tokenize the question and answer pairs. 
tokenizer_path = "/content/tokenizer_pickle" #input('Enter pickled Tokenizer path : ')

# The output directory where the files will be stored.
output_dir = "/content" #input( 'Enter path of output directory : ')

# The output dimensions for the embedding. If the `glove_path` refers to `glove.6B.50d` then this argument should be equal to 50.
output_dim = embedding_dim #int(input('Enter output dimension for Embedding : '))

# The filename for the saved numpy array. For example, "embedding_matrix.npy"
embedding_filename = "embedding_matrix_6b.npy" #input('Enter file name : ' )

# load the whole embedding into memory
embeddings_index = dict()
f = open( glove_path , encoding='utf8' )
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

tokenizer = pickle.load( open( tokenizer_path , 'rb' ) )

embedding_matrix = np.zeros((vocab_size, output_dim))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

np.save( os.path.join( output_dir , '{}'.format( embedding_filename )) , embedding_matrix )

Loaded 400001 word vectors.


In [ ]:
embedding_matrixs =  np.load("/content/embedding_matrix_6b.npy")

In [ ]:
embedding_matrixs.shape

(7516, 300)

In [34]:
# Train model
parameters = {
    'batch_size' : 4 ,
    'validation_split' : None ,
    'epochs' : 1 ,
    'val_data' : None
}
lyrics_generator = LyricsGenerator(embedding_dim, total_words, x_train.shape[1], x_train_midi.shape[1], embedding_matrixs)
x_midi = np.asarray(x_train_midi).astype('float32')
h = lyrics_generator.fit([x_train[:10000], x_midi[:10000]], y_train[:10000], parameters) # TODO: !! remove the limits
# lyrics_generator.plot_metric(h)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1480)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1480, 300)    2254800     input_5[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 300)          721200      embedding_2[0][0]                
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 17)]         0                                            
____________________________________________________________________________________________

In [35]:
# h = lyrics_generator.fit(x_train, x_midi, y_train, parameters)

In [36]:
def generate_text(x, eof, model, max_sequence_len, vocab_size):
    seed_text, midi_metadata = x
    next_word = ""
    word_indices = np.arange(vocab_size) + 1
    word_count = 0
    while next_word != eof and word_count < 500:
        token_list = tokenize.texts_to_sequences([seed_text])[0]
        # print(token_list)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='post')
        # print(token_list)
        predicted_proba = model.model.predict_on_batch([token_list, midi_metadata])
        # print(predicted_proba.reshape((predicted_proba.shape[1])).shape)
        # sample the word index acording to predicted proba
        chosen_index = np.random.choice(word_indices ,1 ,replace=False, p=predicted_proba.reshape((predicted_proba.shape[1])))
        word_count += 1

        
        next_word = ""
        for word, index in tokenize.word_index.items():
            if index == chosen_index:     
                next_word = word
                print(next_word)
                break
        seed_text += " " + next_word
    return seed_text

In [37]:
onehot_encoder.categories_

[array(['major', 'minor', 'unknown'], dtype=object)]

In [38]:
songs_test, df_songs_test = load_songs('/content/drive/MyDrive/DL/Assignment_3/Lyrics/lyrics_test_set.csv')
df_songs_test['filename'] = df_songs_test['artist'].str.replace(' ', "_") + "_-" + df_songs_test['title'].str.replace(' ', "_") + ".mid"
df_merged_test = pd.merge(df_songs_test, df_midi, on="filename")

columns = ['numerator', 'denominator','clocksPerTick', 'demisemiquaverPer24Clocks', 'tempo']
for i, col in enumerate(columns):
  scaled_features = scalers[i].transform(df_merged_test[col].values.reshape(-1, 1))
  tmp_df = pd.DataFrame(scaled_features, index=df_merged_test.index, columns=[col])
  df_merged_test[col] = tmp_df[col]


columns = ['key', 'mode']
for i, col in enumerate(columns):
  scaled_features = onehot_encoders[i].transform(df_merged_test[col].values.reshape(-1, 1))
  tmp_df = pd.DataFrame(scaled_features, index=df_merged_test.index)
  tmp_df.columns = onehot_encoders[i].get_feature_names([col])
  df_merged_test.drop([col] ,axis=1, inplace=True)
  df_merged_test = pd.concat([df_merged_test, tmp_df ], axis=1)

x_test, x_test_midi, y_test, max_sequence_len_train = create_x_y_train(df_merged_test, tokenize, total_words)

[[333, 13], [333, 13, 113], [333, 13, 113, 111], [333, 13, 113, 111, 8], [333, 13, 113, 111, 8, 13], [333, 13, 113, 111, 8, 13, 272], [333, 13, 113, 111, 8, 13, 272, 678], [333, 13, 113, 111, 8, 13, 272, 678, 1], [333, 13, 113, 111, 8, 13, 272, 678, 1, 29], [333, 13, 113, 111, 8, 13, 272, 678, 1, 29, 2]]


In [39]:
x_test_midi[0]

array([0.38461538461538464, 0.5555555555555556, 0.2577319587628866, 1.0,
       0.46742552709198043, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       1.0, 0.0, 0.0], dtype=object)

In [40]:
# np.array(pad_sequences('close', maxlen=max_sequence_len, padding='post'))

In [41]:
x_midi = np.asarray(x_test_midi[0]).astype('float32').reshape(1,x_test_midi[0].size)
print(generate_text(['close', x_midi], "EOF" , lyrics_generator , max_sequence_len, total_words))

silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
silencio
s

KeyboardInterrupt: ignored